In [83]:
import os
import re
import networkx as nx
import xml.etree.ElementTree as ET

In [84]:
def clean_xml_content(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        # Use regex to find the first closing tag of the root element and cut the content there
        match = re.search(r'(</ClinicalTrial>)', content)
        if match:
            end = match.end()
            content = content[:end]
        return content

In [85]:
def extract_combos_from_xml(file_path):
    try:
        content = clean_xml_content(file_path)
        # print(content)
        tree = ET.ElementTree(ET.fromstring(content))
        root = tree.getroot()

        combos = []
        for combo in root.findall('.//Combo'):
            drug1 = combo.get('drug1')
            drug2 = combo.get('drug2')
            combos.append((drug1, drug2))

        return combos
    except ET.ParseError as e:
        # print(f"Error parsing {file_path}: {e}")
        return []

In [109]:
def build_graph_from_directory(directory):
    
    count  = 0
    graph = nx.Graph()
    drugs = []
    for file_name in os.listdir(directory):
        if file_name.startswith("NCT") and file_name.endswith(".xml"):
            nct_id = file_name.split('.')[0]  # Extract NCTXXX from file name
            file_path = os.path.join(directory, file_name)
            combos = extract_combos_from_xml(file_path)

            for drug1, drug2 in combos:
                if drug1 not in drugs and drug2 not in drugs:
                    drugs.append(drug1)
                    drugs.append(drug2)                    
                    
                    count += 1
                    # if not graph.has_edge(drug1, drug2):
                    graph.add_edge(drug1, drug2, label=nct_id)
    print(set(drugs))
    return graph

In [110]:

directory = "../datasets/ft-discovered/well-formatted/"  # Replace with your directory path
# # print(len(directory))
populated_graph = build_graph_from_directory(directory)

{'Hydroxychloroquine', 'zinc sulfate', 'darunavir', 'hydroxychloroquine', 'Ritonavir', 'Zinc Sulfate', 'Darunavir', 'Zinc sulfate', 'Ivermectin', 'ivermectin'}


TypeError: 'data' is an invalid keyword argument for print()